In [2]:

!pip install beautifulsoup4
!pip install requests
!pip install spacy
!pip install trafilatura


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for urllib3<3,>=1.26 from https://files.pythonhosted.org/packages/37/dc/399e63f5d1d96bb643404ee830657f4dfcf8503f5ba8fa3c6d465d0c57fe/urllib3-2.0.5-py3-none-any.whl.metadata
   ---------------------------------------- 123.8/123.8 kB ? eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 2.0.5 which is incompatible.


In [ ]:
from bs4 import BeautifulSoup
import json
import os
import numpy as np
import requests
import re

from requests.models import MissingSchema
import spacy
import trafilatura

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)  #%%




In [ ]:
from bs4 import BeautifulSoup
import json
import os
import numpy as np
import requests
import re

from requests.models import MissingSchema
import spacy
import trafilatura

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
urls = ['https://website.understandingdata.com/',
        'https://sempioneer.com/', ]
data = {}

for url2 in urls:
    # 1. Obtain the response:
    resp = requests.get(url2)

    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url2] = resp.text


In [ ]:
def beautifulsoup_extract_text_fallback(response_content):
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style', ]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def parse_page(link):
    downloaded_url = trafilatura.fetch_url(link, no_ssl=True)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return downloaded_url, json_output['text']
    else:
        try:
            resp = requests.get(link)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:

                all_relative = re.findall('href=(\'|")(.*?)(\'|")', resp.content.decode('utf-8'))
                for aa in all_relative:
                    if 'http' in aa[1]:
                        continue
                    push_url('http://fsb.ru/' + aa[1])

                all_abs = re.findall('href=(\'|")(http.*?)(\'|")', resp.content.decode('utf-8'))
                for aa in all_relative:
                    if not 'http' in aa[1]:
                        continue
                    push_url(aa[1])

                return resp.content.decode('utf-8'), beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return "", np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return "", np.nan


urls = []
done = []


def push_url(url):
    global urls
    if url not in done:
        print(f'get {url} ... [{len(done)}]')
        page, text = parse_page(url)

        done.append({'url': url, 'page': page, 'text': text})


single_url = 'http://www.fsb.ru/'
page, text = parse_page(single_url)
print(urls)

In [5]:
urls = ['https://website.understandingdata.com/',
        'https://sempioneer.com/', ]
data = {}

for url2 in urls:
    # 1. Obtain the response:
    resp = requests.get(url2)

    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url2] = resp.text


In [28]:
def beautifulsoup_extract_text_fallback(response_content):
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style', ]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def parse_page(link):
    downloaded_url = trafilatura.fetch_url(link, no_ssl=True)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError as e:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
        print(f'Except Attribute {e}')
    if a:
        json_output = json.loads(a)

        parse(downloaded_url)

        return downloaded_url, json_output['text']
    else:
        try:
            resp = requests.get(link)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:

                parse(resp)

                return resp.content.decode('utf-8'), beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return "", np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema as e:
            print(f'MissingSchema {e}')
            return "", np.nan


def parse(responce):
    all_relative = re.findall('href=(\'|")(.*?)(\'|")', responce.content.decode('utf-8'))
    for aa in all_relative:
        if 'http' in aa[1]:
            continue
        push_url('http://fsb.ru/' + aa[1])
    all_abs = re.findall('href=(\'|")(http.*?)(\'|")', responce.content.decode('utf-8'))
    for aa in all_relative:
        if not 'http' in aa[1]:
            continue
        push_url(aa[1])


urls = []
done = []
done_urls = []


def push_url(url):
    global urls, done_urls
    if url not in done_urls:
        print(f'get {url} ... [{len(done)}]', end='')
        #print(f'doneurls=[{done_urls}]')

        try:
            page, _ = parse_page(url)
        except Exception as e:
            print(f'exception: {e}')

        print(f'{len(page)} bytes.')
        done.append({'url': url, 'page': page, 'text': text})
        done_urls.append(url)


single_url = 'http://www.fsb.ru/'
page, text = parse_page(single_url)
print(urls)


get http://fsb.ru/fsb/rss.htm%21contenttype%3Dxml.html ... [0]

F:\fux-issues-bot\venv\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if sys.path[0] == "":


0 bytes.
get http://fsb.ru/styles/fsb2019/bootstrap.min.css ... [1]121411 bytes.
get http://fsb.ru/styles/fsb2019/all.css ... [2]30834 bytes.
get http://fsb.ru/styles/fsb/ie_fix.css ... [3]170 bytes.
get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]get http://fsb.ru/fsb.htm ... [4]

KeyboardInterrupt: 